# Dual Rail USB-C PSU

The general idea is to create a reasonably compact and efficient 12V dual rail PSU suitable for a Eurorack setup.
* Deliver +/-12VDC at 1A (24W)
* Test at 150% load sustained
* Powered from a USB-C connection to a 60W adapter (minimum, capable of delivering 15V @ 3A)

A secondary goal is to learn more about
* switching mode power supplies
* USB-C power delivery (PD)

# Notes

![Block diagram](./images/block_diagram.png)

## USB Type C Power Delivery (PD) Sink
* Sink only, “no data” role, PD
* Primer: https://www.ti.com/lit/wp/slyy109b/slyy109b.pdf
* ST notes: https://www.st.com/content/ccc/resource/sales_and_marketing/presentation/product_presentation/group0/5a/b1/8e/6c/2b/0d/46/3c/Apec/files/APEC_2016_USB_Power.pdf/_jcr_content/translations/en.APEC_2016_USB_Power.pdf

* Target: 3A supply at up to 20V (60W)
    * Request 15V @ 3A = 45W

### Parts
* HUSB238 - Hynetek - https://en.hynetek.com/2421.html
    * https://www.adafruit.com/product/5807
* RT1719 - Richtek - https://www.richtek.com/Products/USB%20PD%20IF/USB%20Type-C%20and%20Power%20Delivery/RT1719?sc_lang=en
* STUSB4500 - ST - https://www.st.com/en/interfaces-and-transceivers/stusb4500.html
* AP33771 - Diodes Inc -https://www.diodes.com/assets/Datasheets/AP33771.pdf 
* TPS25730 - TI - https://www.ti.com/lit/ds/symlink/tps25730.pdf
* CYPD3177 - Infineon - https://www.infineon.com/cms/en/product/universal-serial-bus/usb-c-charging-port-controllers/ez-pd-barrel-connector-replacement-bcr/cypd3177-24lqxqt/

## Negative Rail: Inverting Buck SMPS - Cuk + LDO 
* Input: 15V @ 3A or 20V @ 3A
* SMPS
    * Topology: https://en.wikipedia.org/wiki/%C4%86uk_converter
    * Output: -12V - 2V_DO,max at 2A max
    * TDP @ 85% efficiency is 4W (expecting 3W at 1.5A)
    * Part: LT3757A
        * alternatives: LT3959 (integrated 6A switch, stable with MLCC), synchronous (reduce ID low enough)
* LDO 
    * VDO,max = 0.76V
    * Output = -12V (fixed) at 1.5A max
    * TDP ~ 1.5*1.5 = 2.25W
    * Part = L7912CV

## Positive Rail: LDO
* TDP ~ 3V*1.5A = 4.5W, compatible with a L7812CV LDO


# Negative Rail Design

Datasheet: [LT3757](https://www.analog.com/media/en/technical-documentation/data-sheets/lt3757-3757a.pdf)

![Typical application](./images/typical_app_inverting_converter.png)

Other reference parts: LT3580 (internal switch)

![Typical layout](./images/typical_layout_inverting_mutual_L.png)

![Current paths](./images/current_paths_inverting_topo.png)

![Demo board](./images/demo_board.png)

## Useful references
* [Buck converter design](https://www.mouser.com/pdfdocs/buckconverterdesignnote.pdf)


### Design Parameters

* Input power: 45W
    * USB-C PD at 3A provides 45W at 15V or 60W at 20V. 
    * Initial design for 45W total for all rails.
* Output power: 36W (80%) 
    * Each 12V rail (positive & negative) can provide 15W (30W total), equivalent to 1.25A
    * 5V rail provides 1.25A (6W)

In [2]:
import math

VIN_MAX = 20.0      # nominal: protect against spikes to 28V
VIN_MIN = 15.0
VNEG_DROP_OUT = -2
VNEG_OUT = -12.0    
VOUT = VNEG_OUT + VNEG_DROP_OUT 
delta_VOUT = 0.005  # output ripple pk-pk 
IO_max = 1.6        # 15W*1.5 = 22.5W / 14V = 1.6A (50% headroom)
T_amb = 70          # deg 
P_out = abs(VOUT*IO_max)

diode_part = "SDT5A60SA-13"
inductor_part = "104CDMCCDS-330M" #"CYA1040-33UH" #"CYA0650-15UH" 
MOSFET_part = "SIR184DP-T1-RE3" #"SIS4608DN" #"BRCS4266SC"

print(f"Negative rail SMPS")
print(f" + Output voltage: {VOUT:.3g}V")
print(f" + Max. output power: {P_out:.3g}W")
print(f" + Max. output current: {IO_max:.3}A")


Negative rail SMPS
 + Output voltage: -14V
 + Max. output power: 22.4W
 + Max. output current: 1.6A


### Switching Regulator
The LT3757 is a non-synchronous SMPS controller with an external switch and a negative voltage capable feedback suitable for an inverting (Cuk) power supply.
* Input voltage range from 2.9-40V
* Internal oscillator with switching frequency programmable from 100kHz to 1MHz

In [3]:
#
# SMPS Controller: LT3757
#
f_SWITCH = 1e6
MIN_ON_OFF_T = 0.22e-6

#
# Diode selection for VD
#
if diode_part == "30BQ060": # Diode characteristics (from 30BQ060HM3/9AT)
    R_thJA_D = 46       # degC/W, steady state max.
    VD = 0.52
elif diode_part == "SSL56F":
    R_thJA_D = 45       # degC/W, steady state max.
    VD = 0.50
elif diode_part == "SDT5A60SA-13":
    R_thJA_D = 56
    VD = 0.46

### Duty Cycle
At steady-state, the average voltage across the inductors L1 and L2 must be zero [ref](https://en.wikipedia.org/wiki/%C4%86uk_converter#Continuous_mode). In the switch-on phase, the switch node (SW) of L1 is pulled to ground, so $V_{L1} = V_{IN}$ and $V_{L2} = V_{OUT} - (-V_{C2})$. In the switch-off phase, the rectifying diode pulls node SWX to $V_D$ such that $V_{L1} = V_{IN} - V_{C2} - V_D$ and $V_{L2} = V_{OUT} - V_D$ (note: $V_{OUT} < 0$). Averging over a full switching cycle, 
$$
\begin{align*}
0 = V_{L1} &= DV_{IN} + (1-D)(V_{IN}-V_{C2}-V_D) = V_{IN} - (1-D)(V_{C2}+V_{D})\\
0 = V_{L2} &= D(V_{OUT} + V_{C2}) + (1-D)(V_{OUT}-V_D) = V_{OUT} + D(V_{C2}+V_D) - V_D
\end{align*}
$$
Eliminating $V_{C2} + V_D$ (multiply the equation for $V_{L1}$ by $D/(1-D)$ and sum):
$$
\begin{align*}
 0 &= \frac{DV_{IN}}{1-D} - D(V_{C2} + V_{D})\\
+\quad 0 &=  V_{OUT} + D(V_{C2}+V_D) - V_D \\
\to 0 &= \frac{DV_{IN}}{1-D} + V_{OUT} -V_D \\
\to 0 &= DV_{IN} + (1-D)(V_{OUT} - V_D) \\
\to D &= \frac{V_{OUT} - V_D}{V_{OUT}-V_D-V_{IN}}
\end{align*}
$$
Therefore
$$
\begin{align*}
D_{max} &= \frac{V_{OUT} - V_D}{V_{OUT}-V_D-V_{IN,min}}\\
D_{min} &= \frac{V_{OUT} - V_D}{V_{OUT}-V_D-V_{IN,max}}
\end{align*}
$$


In [4]:
#
# Duty cycle
#
DMIN_CLK = f_SWITCH*MIN_ON_OFF_T
DMAX_CLK = 1.0 - f_SWITCH*MIN_ON_OFF_T
DMAX = (VOUT-VD)/(VOUT - VD - VIN_MIN)
DMIN = (VOUT-VD)/(VOUT - VD - VIN_MAX)
print(f"Duty cycle (D)")
print(f" + Max. duty cycle (DMAX): {DMAX:.2g} (max. from clock is {DMAX_CLK:.2g})")
print(f" + Min. duty cycle (DMIN): {DMIN:.2g} (max. from clock is {DMIN_CLK:.2g})")
print()

Duty cycle (D)
 + Max. duty cycle (DMAX): 0.49 (max. from clock is 0.78)
 + Min. duty cycle (DMIN): 0.42 (max. from clock is 0.22)




### Inductor Selection
Assuming the ideal lossless case, $-P_{OUT} = P_{IN}$. The current in L2 supplies the load at $V_{OUT}$, so 
$$
I_{L2} = I_{O} = \frac{P_{OUT}}{V_{OUT}}
$$
$V_{OUT}$ and $V_{IN}$ are related through the duty cycle (assuming $V_D=0$ for convenience), $V_{OUT} = -V_{IN}\frac{D}{1-D}$ so
$$
P_{IN} = I_{IN}V_{IN} = I_{L1}V_{IN} = -I_{O}V_{OUT} = -I_{O}V_{IN}\frac{D}{1-D} \to I_{L1} = I_{O}\frac{D}{1-D}
$$
These currents are at their maximums when the output current is at its maximum and $D=D_{max}$:
$$
\begin{align*}
I_{L2,max} &= I_{O,max} \\
I_{L1,max} &= I_{O,max}\frac{D_{max}}{1-D_{max}}
\end{align*}
$$
The inductor ripple current is a design parameter. To size the inductors, consider the maximum current in the switch, which combines the two inductor currents: $I_{SW,max} = I_{L1,max} + I_{L2,max}$. Define the switch ripple current as $\Delta I_{SW} = \chi I_{SW,max}$, where the target fractional ripple current is $\chi=0.2$. The inductor currents are in phase, so $\Delta I_{L1} = \Delta I_{L2} = 0.5\Delta I_{SW}$. 

Using the relationship $V_L = L \frac{dI_L}{dt} \simeq L \frac{\Delta I_{Lx}}{D T}$, where $T=\frac{1}{f_s}$ and $f_s$ is the switching frequency, the inductor size can be estimated. 
$$
L = \frac{2 D V_{IN}}{f_s \Delta I_{SW}} = \frac{2 D V_{IN}}{f_s \chi I_{O,max}(1 + \frac{D}{1-D})} = \frac{2 D (1-D) V_{IN}}{f_s \chi I_{O,max}}
$$
As $V_{IN}$ increases, $D$ decreases, but not at the same rate. Consequently, larger inductors are estimated for the upper end of the input voltage range. However, in the datasheets, $D_{max}$ and $V_{IN,min}$ are used when estimating the inductor sizing. In this design, we assume that the input voltage is set by the USB-C PD standard, so 
$$
L = \frac{2 D_{max} V_{IN,min}}{f_s \Delta I_{SW}} = \frac{2 D_{max} V_{IN,min}}{f_s \chi I_{SW,max}}
$$
where $I_{SW,max} = I_{O,max}/(1-D_{max})$.

Note: a more negative $V_{OUT}$ will slightly increase $D_{max}$, but I assume it will have a negligible impact on ripple (confirmed for $V_{OUT}=-14V$). Therefore, it's OK to adjust $V_{OUT}$ to a more negative value if an LDO is desired to reduce ripple/noise.





In [5]:
# Inductor characteristics
if inductor_part in ["B82477D4153M000", "SRF1260-150M", "DRQ127-150-R"]:
    L_mutual_uH = 15
    L_discrete_uH = None
elif inductor_part in ["CYH127-33UH", "CYA1040-33UH", "104CDMCCDS-330M"]:
    L_mutual_uH = None
    L_discrete_uH = 33
elif inductor_part == "CYH127-22UH":
    L_mutual_uH = None
    L_discrete_uH = 22
elif inductor_part == "CYA0650-15UH":
    L_mutual_uH = None
    L_discrete_uH = 15

chi = 0.2   # can be overriden when selecting L

#
# Inductor and Sense Resistor
#
I_L1_max = IO_max * (DMAX/(1-DMAX))
I_L2_max = IO_max
I_SW_max = I_L1_max + I_L2_max

if L_mutual_uH is not None:
    I_SW_ripple = VIN_MIN/L_mutual_uH/f_SWITCH * DMAX * 1e6
    chi = I_SW_ripple/I_SW_max
elif L_discrete_uH is not None:
    # nominally V_IN across L2 --> V_IN = L2 di2/dt = L2*I_L2_ripple/DMAX.T
    I_SW_ripple = 2*VIN_MIN*DMAX/L_discrete_uH/f_SWITCH * 1e6
    chi = I_SW_ripple/I_SW_max
else: 
    I_SW_ripple = chi * I_SW_max

I_SW_peak = (1 + 0.5*chi)*IO_max/(1-DMAX)

I_L1_L2_ripple = 0.5*I_SW_ripple
I_L1_peak = I_L1_max + 0.5*I_L1_L2_ripple
I_L2_peak = I_L2_max + 0.5*I_L1_L2_ripple

chi_L1 = I_L1_L2_ripple/I_L1_max
chi_L2 = I_L1_L2_ripple/I_L2_max
I_L1_RMS = I_L1_max*math.sqrt(1 + chi_L1*chi_L1/12)
I_L2_RMS = I_L2_max*math.sqrt(1 + chi_L2*chi_L2/12)

L_common_uH = VIN_MIN/I_SW_ripple/f_SWITCH * DMAX * 1e6
L1_L2_uH = 2*VIN_MIN/I_SW_ripple/f_SWITCH * DMAX * 1e6
R_sense = 0.08 / I_SW_peak

print(f"Inductors and Sense Resistor")
print(f" + I_(Lx,ripple) = {I_L1_L2_ripple:.3g} A")
print(f" + chi = {chi:0.3g} (chi_L1={chi_L1:0.3g}, chi_L2={chi_L2:0.3g})")
print(f" + L_(common) = {L_common_uH:.3g} uH for f_SWITCH={f_SWITCH/1000} kHz")
print(f" + L1 = L2 = {L1_L2_uH:.3g} uH for f_SWITCH={f_SWITCH/1000} kHz")
print()
print(f"        |  max (A) | peak (A) |  RMS (A) ")
print(f"   -----+----------+----------+----------")
print(f"   I_L1 | {I_L1_max:8.3g} | {I_L1_peak:8.3g} | {I_L1_RMS:8.3g} ")
print(f"   I_L2 | {I_L2_max:8.3g} | {I_L2_peak:8.3g} | {I_L2_RMS:8.3g} ")
print()
print(f" + I_(SW,max) = {I_SW_max:.3g} A")
print(f" + I_(SW,peak) = {I_SW_peak:.3g} A")
print(f" + R_(sense) = {R_sense:.3g} Ohm")
print(f" + P_(sense) = {I_SW_peak*I_SW_peak*R_sense:.3g} W")
print()


Inductors and Sense Resistor
 + I_(Lx,ripple) = 0.223 A
 + chi = 0.142 (chi_L1=0.145, chi_L2=0.139)
 + L_(common) = 16.5 uH for f_SWITCH=1000.0 kHz
 + L1 = L2 = 33 uH for f_SWITCH=1000.0 kHz

        |  max (A) | peak (A) |  RMS (A) 
   -----+----------+----------+----------
   I_L1 |     1.54 |     1.65 |     1.54 
   I_L2 |      1.6 |     1.71 |      1.6 

 + I_(SW,max) = 3.14 A
 + I_(SW,peak) = 3.37 A
 + R_(sense) = 0.0238 Ohm
 + P_(sense) = 0.269 W



In [6]:
import math


# MOSFET characteristics (from SI7850DP)
if MOSFET_part == "SI7850DP":
    RDS_ON = 0.031      # Ohm
    C_RSS = 5.3e-9/20   # F
    R_thJA = 70         # degC/W, steady state max.
elif MOSFET_part == "SIS4608LDN":  # LDN is 4.5V gate drive, slower rise/fall 
    RDS_ON = 0.015
    C_RSS = 6e-12
    R_thJA = 38
elif MOSFET_part == "SIS4608DN":    # DN is 7.5V gate drive
    RDS_ON = 0.015
    C_RSS = 7e-12
    R_thJA = 38
elif MOSFET_part == "SIR184DP-T1-RE3":   
    RDS_ON = 0.007
    C_RSS = 22e-12
    R_thJA = 25
elif MOSFET_part == "BRCS4266SC":
    RDS_ON = 0.015
    C_RSS = 10e-12
    R_thJA = 75
elif MOSFET_part == "PJQ5466A":
    RDS_ON = 0.021
    C_RSS = 85e-12
    R_thJA = 62.5
elif MOSFET_part == "06N06L":
    RDS_ON = 0.040
    C_RSS = 40e-12
    R_thJA = 130

#
# Power MOSFET
#
P_FET = I_SW_max*I_SW_max*RDS_ON*DMAX + 2*(VIN_MIN + VOUT)**2 * I_SW_max * C_RSS * f_SWITCH
print(f"Power MOSFET")
print(f" + P_(FET) = {P_FET:.2g} W")
print(f" + T_(J) = {T_amb + P_FET*R_thJA:.3g} deg. C")
print()

#
# Output Diode
#
ID_peak = (1 + 0.5*chi) * IO_max / (1-DMAX)
PD = IO_max * VD
print(f"Output Diode")
print(f" + V_(D,fwd) = {VD:.2g} V")
print(f" + I_(D,peak) = {ID_peak:.2g} A")
print(f" + P_(D) = {PD:.2g} W")
print(f" + T_(J) = {T_amb + PD*R_thJA_D:.3g}")
print()

#
# DC coupling Capacitor CDC
#
IRMS_CDC = IO_max * math.sqrt(DMAX / (1-DMAX))
V_CDC_MIN = VIN_MAX - VOUT

print(f"C_DC")
print(f" + Voltage min. rating: {V_CDC_MIN:.2g} V")
print(f" + RMS current: {IRMS_CDC:.2g} A")
print()

#
# Output capacitor 
#
C_out = I_L1_L2_ripple/8/f_SWITCH/delta_VOUT
print(f"C_out")
print(f"  + C_out: {C_out*1e6:.3g} uF")
print(f"  + ESR(C_out): {delta_VOUT/ID_peak*1000:.3g} mOhm")

Power MOSFET
 + P_(FET) = 0.034 W
 + T_(J) = 70.9 deg. C

Output Diode
 + V_(D,fwd) = 0.46 V
 + I_(D,peak) = 3.4 A
 + P_(D) = 0.74 W
 + T_(J) = 111

C_DC
 + Voltage min. rating: 34 V
 + RMS current: 1.6 A

C_out
  + C_out: 5.58 uF
  + ESR(C_out): 1.49 mOhm


## Pin Configuration
### UVLO
Undervoltage lockout is programmed with a resitive divider, and an internal 2uA current source provides hysteresis. Set the UVLO threshold to 8V.
$$
V_{VIN,falling} = 8 = 1.22\frac{R_3 + R_4}{R_4} \to \frac{R_3}{R_4} = \frac{8}{1.22} - 1 = 5.56
$$
With hysteresis, the UVLO is disabled when $V_{IN}$ rises above $V_{VIN,rising} = 2\mu\mathrm{A}\times R_3 + V_{VIN,falling}$. Setting the hysteresis to 0.36V results in $R_3=180\mathrm{k}\Omega$ and $R_4 = \frac{180\mathrm{k}}{5.56} \approx 33\mathrm{k}\Omega$, and $V_{VIN,falling} = 7.87V$. 

### INTVCC
The LT3757A includes an interal 7.2V LDO (400mV dropout voltage). This supplies the gate driver for the external switch. The driver is limited by maximum junction temperature rating. Using the formula provided in the datasheet
$$
T_J = T_A + V_{IN}(I_Q + f Q_G)\theta_{JA}
$$
where $T_A=70$ is the reference ambient temperature, $I_Q=1.6\mathrm{mA}$ is the quiescent current, $f$ is the switching frequency, $Q_G$ is the gate charge of the external MOSFET, and $\theta_{JA}$ is the thermal conductivity of the LT3757 package. The limiting gate charge is then
$$
fQ_G \leq \frac{T_J - T_A}{\theta_{JA} V_{IN}}-I_Q
$$
The internal LDO must have a minimum 4.7uF bypass capacitor located adjacent to the INTVCC pin.

In [7]:
#
# limiting gate charge
#  
LT3757 = {'TJ_max':125, 'theta_JA':40, 'IQ':0.0016}
QG_max = ( (LT3757['TJ_max'] - T_amb)/(LT3757['theta_JA']*VIN_MIN) - LT3757['IQ'] ) / f_SWITCH
print(f'Max MOSFET gate charge: {QG_max*1e9:0.3g}nC')

Max MOSFET gate charge: 90.1nC


### Soft Start (SS)
The capacitor on the SS pin is fed with a 10uA source. SS is internally pulled low on UVLO. When the supply is at an adequate level, SS rises towards 2.5V. Use a capacitor $C_{SS}$ per the data sheet (100nF typical).

### Switching Frequency Adjust (RT)
The value of this resistor is selected to set the oscillator frequency (see the table in the datasheet). $R_T=10.5k\Omega$ will set the operating frequency to 1MHz.

### Error Amplifier Compensation (VC)
Stabilize the external loop with an RC filter. Values were adjust based on simulaton with LTSpice: $R=22k\Omega$ in series with $C=6.8nF$ and $C=100pF$ in parallel to that branch.


# 5V Buck Regulator

Part: AP63205

![AP63205](./images/AP630205_typical_app.png) 

![Recommend component values](./images/AP63205_recommended_components.png)

### Inductor Selection

$$
L = \frac{V_{OUT}(V_{IN} - V_{OUT})}{V_{IN}\Delta I_L f_{sw}}
$$
with $V_{OUT}=5V$, $V_{IN,nom.}=15V$, and $f_{sw}=1.1\mathrm{MHz}$. $\Delta I_L$ is the inductor ripple current; choose the ripple current as $\Delta I_L = 0.2I_{L,max} = 0.2\times 2=0.4A$. 


In [8]:
class BuckReg:
    def __init__(self,Vin):
        self.Vout = 5
        self.fsw = 1.1e6
        self.Imax = 2
        self.IL_ripple = 0.2*self.Imax
        self.L = self.Vout*(Vin - self.Vout)/Vin/self.IL_ripple/self.fsw
        self.IL_peak = self.Imax + 0.5*self.IL_ripple

    def __str__(self):
        return f"AP63205\n + L > {self.L*1e6:.3g}uH\n + I(L,peak) = {self.IL_peak:.3g}A"

print(BuckReg(15))

AP63205
 + L > 7.58uH
 + I(L,peak) = 2.2A


Choose a 10uH inductor with $I_{sat.} > 3A$: 0624CDMCCDS-100MC